In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
%pylab inline
from fastai.vision import *
from fastai import  *
from pathlib import Path
import pandas as pd
import torch
import pickle
import cv2
import time 

Populating the interactive namespace from numpy and matplotlib


In [3]:
%cd '/content/gdrive/My Drive/Colab Notebooks/deep_learning/bbc_train'

/content/gdrive/My Drive/Colab Notebooks/deep_learning/bbc_train


In [4]:
cur_dir = Path.cwd() 
cur_dir

PosixPath('/content/gdrive/My Drive/Colab Notebooks/deep_learning/bbc_train')

In [5]:

path=Path('/content/gdrive/My Drive/Colab Notebooks/deep_learning/bbc_train')

In [ ]:
labels=json.load(open('bboxes.json', 'r'))

In [ ]:
images, lbl_bbox = get_annotations('bboxes.json')

In [19]:
def get_y_fn(x): return path/'masks'/(x.stem + '.png')
classes = array(['background', 'ball', 'brick', 'cylinder'])

In [20]:
src = (SegmentationItemList.from_folder(str(path) + "/images/")
       .split_by_rand_pct(0.2)
       .label_from_func(get_y_fn, classes=classes))


In [ ]:
labels

{'annotations': [{'area': 0,
   'bbox': [254.82692307692304,
    145.47435897435898,
    92.62820512820514,
    91.66666666666666],
   'category_id': 2,
   'id': 1,
   'ignore': 0,
   'image_id': 'P2050936',
   'iscrowd': 0,
   'segmentation': [1178.2820512820513,
    587.025641025641,
    1106.4871794871794,
    615.2307692307692,
    1066.7435897435896,
    648.5641025641025,
    1035.974358974359,
    696.0,
    1019.3076923076922,
    757.5384615384615,
    1025.7179487179487,
    798.5641025641025,
    1041.102564102564,
    854.9743589743589,
    1082.128205128205,
    903.6923076923076,
    1159.051282051282,
    948.5641025641025,
    1218.0256410256409,
    946.0,
    1291.102564102564,
    922.9230769230769,
    1325.7179487179487,
    904.9743589743589,
    1375.7179487179487,
    849.8461538461538,
    1389.8205128205127,
    779.3333333333333,
    1379.5641025641025,
    711.3846153846154,
    1337.2564102564102,
    640.8717948717948,
    1274.4358974358975,
    596.0,
  

In [21]:
data = (src.transform(get_transforms(), tfm_y=True, size=(256,256))
        .databunch(bs=16)
        .normalize(imagenet_stats))

In [ ]:
def conv_trans(ni, nf, ks = 4, stride = 2, padding = 1):
  return nn.Sequential(
        nn.ConvTranspose2d(ni, nf, kernel_size=4, bias=False, stride=2, padding = 1), 
        nn.ReLU(inplace = True), 
        nn.BatchNorm2d(nf))

In [9]:
def class_pred_accuracy(yhat, y):
    y_nump = y.cpu().detach().numpy()
    y_nump_unique = [np.unique(y_nump[i][y_nump[i]!=0])[0] for i in range(y_nump.shape[0])]
    class_labels = torch.tensor(y_nump_unique).to('cuda')
    return accuracy(yhat[0][:,4:], class_labels.view(-1))

In [10]:
def bbox_accuracy(yhat, y):
    bboxes = torch.zeros((y.shape[0], 4))

    y_nump = y.cpu().detach().numpy()
    for i in range(y_nump.shape[0]):
        rows,cols = np.where(y_nump[i, 0]!=0)
        bboxes[i, :] = torch.tensor([rows.min(), cols.min(), rows.max(), cols.max()])

    return nn.L1Loss()(yhat[0][:,:4], bboxes.to('cuda'))


In [13]:
def pixel_accuracy(yhat, y):
    y_ = y.squeeze(dim=1)
    yhat_ = yhat[1].argmax(dim=1)
    return (y_ == yhat_).sum().float()/y.numel()


In [12]:
class Loss(nn.Module):
  def forward(self, yhat, y):
    y_nump = y.cpu().detach().numpy()
    y_nump_unique = [np.unique(y_nump[i][y_nump[i]!=0])[0] for i in range(y_nump.shape[0])]
    class_labels=torch.tensor(y_nump_unique).to('cuda')
        
    bboxes=torch.zeros((y.shape[0], 4))
    for i in range(y_nump.shape[0]):
      rows,cols= np.where(y_nump[i, 0]!=0)
           
      bboxes[i, :] = torch.tensor([rows.min(), cols.min(), rows.max(), cols.max()])

        # Adding all the losses for preduction, segmentation and bounding boxes
    return (nn.CrossEntropyLoss()(yhat[0][:,4:], class_labels.view(-1))) + (nn.L1Loss()(yhat[0][:,:4], bboxes.to('cuda'))) + (nn.CrossEntropyLoss()(yhat[1],y.squeeze(dim=1)))

In [15]:
from fastai.vision import *
from fastai import  *
from pathlib import Path
import pandas as pd
import torch
import pickle
import cv2
import json
import numpy as np
import torch.nn as nn
import numpy as np


def get_y_fn(x): 
    return path/'masks'/(x.stem + '.png')


class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.detection = nn.Sequential(
                            AdaptiveConcatPool2d((4,4)),
                            Flatten(),
                            nn.ReLU(),
                            nn.Dropout(0.5),
                            nn.Linear(16384,256),
                            nn.ReLU(),
                            nn.BatchNorm1d(256),
                            nn.Dropout(0.5),
                            nn.Linear(256,8))
        
        self.segmentation = nn.Sequential(
                          nn.ConvTranspose2d(512, 256, kernel_size=4, bias=False, stride=2, padding = 1), 
                          nn.ReLU(inplace = True), 
                          nn.BatchNorm2d(256),
                          nn.ConvTranspose2d(256, 128, kernel_size=4, bias=False, stride=2, padding = 1), 
                          nn.ReLU(inplace = True), 
                          nn.BatchNorm2d(128),
                          nn.ConvTranspose2d(128, 64, kernel_size=4, bias=False, stride=2, padding = 1), 
                          nn.ReLU(inplace = True), 
                          nn.BatchNorm2d(64),
                          nn.ConvTranspose2d(64, 32, kernel_size=4, bias=False, stride=2, padding = 1), 
                          nn.ReLU(inplace = True), 
                          nn.BatchNorm2d(32),
                          nn.ConvTranspose2d(32, 4, kernel_size=4, bias=False, stride=2, padding = 1))
        
    def forward(self,x):
        return self.detection(x), self.segmentation(x)
    

class Loss(nn.Module):
    def forward(self, yhat, y):
        y_nump = y.cpu().detach().numpy()
        y_nump_unique = [np.unique(y_nump[i][y_nump[i]!=0])[0] for i in range(y_nump.shape[0])]
        class_labels=torch.tensor(y_nump_unique).to('cuda')
        
        bboxes=torch.zeros((y.shape[0], 4))
        for i in range(y_nump.shape[0]):
            rows,cols= np.where(y_nump[i, 0]!=0)
           
            bboxes[i, :] = torch.tensor([rows.min(), cols.min(), rows.max(), cols.max()])

        # Adding all the losses for preduction, segmentation and bounding boxes
        return (nn.CrossEntropyLoss()(yhat[0][:,4:], class_labels.view(-1))) + (nn.L1Loss()(yhat[0][:,:4], bboxes.to('cuda'))) + (nn.CrossEntropyLoss()(yhat[1],y.squeeze(dim=1)))

def class_pred_accuracy(yhat, y):
    y_nump = y.cpu().detach().numpy()
    y_nump_unique = [np.unique(y_nump[i][y_nump[i]!=0])[0] for i in range(y_nump.shape[0])]
    class_labels = torch.tensor(y_nump_unique).to('cuda')
    return accuracy(yhat[0][:,4:], class_labels.view(-1))

def bbox_accuracy(yhat, y):
    bboxes = torch.zeros((y.shape[0], 4))

    y_nump = y.cpu().detach().numpy()
    for i in range(y_nump.shape[0]):
        rows,cols = np.where(y_nump[i, 0]!=0)
        bboxes[i, :] = torch.tensor([rows.min(), cols.min(), rows.max(), cols.max()])

    return nn.L1Loss()(yhat[0][:,:4], bboxes.to('cuda'))

def pixel_accuracy(yhat, y):
    y_ = y.squeeze(dim=1)
    yhat_ = yhat[1].argmax(dim=1)
    return (y_ == yhat_).sum().float()/y.numel()


class Model(object):
    def __init__(self, path='/content/gdrive/My Drive/Colab Notebooks/deep_learning/Assignment', file='export.pkl'):
        
        self.learn=load_learner(path=path, file=file) #Load model
        self.class_names=['background', 'brick', 'ball', 'cylinder'] #Be careful here, labeled data uses this order, but fastai will use alphabetical by default!

    def predict(self, x):
        '''
        Input: x = block of input images, stored as Torch.Tensor of dimension (batch_sizex3xHxW), 
                   scaled between 0 and 1. 
        Returns: a tuple containing: 
            1. The final class predictions for each image (brick, ball, or cylinder) as a list of strings.
            2. Upper left and lower right bounding box coordinates (in pixels) for the brick ball 
            or cylinder in each image, as a 2d numpy array of dimension batch_size x 4.
            3. Segmentation mask for the image, as a 3d numpy array of dimension (batch_sizexHxW). Each value 
            in each segmentation mask should be either 0, 1, 2, or 3. Where 0=background, 1=brick, 
            2=ball, 3=cylinder. 
        '''

        #Normalize input data using the same mean and std used in training:
        x_norm=normalize(x, torch.tensor(self.learn.data.stats[0]), 
                            torch.tensor(self.learn.data.stats[1]))


        #Pass data into model:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        with torch.no_grad():
            yhat=self.learn.model(x_norm.to(device))

            yhat_class = yhat[0][:,4:]
            bbox = yhat[0][:,:4].cpu().numpy()

            yhat_segment = yhat[1].argmax(1).cpu().detach().numpy()
            im = np.squeeze(yhat_segment)

            yhat_class=yhat_class.detach().cpu()

        #Post-processing/parsing outputs, here's an example for classification only:
        class_prediction_indices=yhat_class.argmax(dim=1)
        class_predictions=[self.class_names[i] for i in class_prediction_indices]

        #Create random segmentation mask:
        mask=yhat_segment
        return (class_predictions, bbox, mask)



In [ ]:
def evaluate(data_path='/content/gdrive/My Drive/Colab Notebooks/deep_learning/bbc_train',
             model_dir = '/content/gdrive/My Drive/Colab Notebooks/deep_learning/Assignment',
             batch_size=64, 
             im_size=(256,256)):

    
    print("1. Loading Data...")

    path=Path(data_path)
    def get_y_fn(x): return path/'masks'/(x.stem + '.png')
    classes = array(['background', 'brick', 'ball', 'cylinder'])

    src = (SegmentationItemList.from_folder(str(path) + "/images/")
       .split_by_rand_pct(0.2)
       .label_from_func(get_y_fn, classes=classes))


    #Don't normalize data here - assume normalization happens inside of Model. 
    data = src.transform(get_transforms(), tfm_y=True, size=im_size).databunch(bs=batch_size)
    print(data)


    print("2. Instantiating Model...")
    M = Model(path=model_dir, file='export.pkl')


    print("3. Evaluating on Test Images...")
    x,y = data.one_batch()
    #Extract class label from mask:
    class_labels=np.array([np.unique(y[i][y[i]!=0])[0] for i in range(x.shape[0])])

    #Extract bounding box from mask:
    bboxes=np.zeros((x.shape[0], 4))
    for i in range(x.shape[0]):
        rows,cols= np.where(y[i, 0]!=0)
        bboxes[i, :] = np.array([rows.min(), cols.min(), rows.max(), cols.max()])

    preds=M.predict(x)

    classification_accuracy=(np.array([data.classes[i] for i in class_labels])==np.array(preds[0])).sum()/len(preds[0])
    bbox_score = 1 - np.mean(np.abs(bboxes-preds[1]))/(x.shape[-1]) #Divide by image height to rougly normalize score
    segmentation_accuracy=float((preds[2] == np.array(y.squeeze(1))).sum())/y.numel()
    combined_accuracy=np.mean([classification_accuracy, bbox_score, segmentation_accuracy])

    return combined_accuracy, classification_accuracy, bbox_score, segmentation_accuracy


def calculate_score(combined_accuracy):
    if combined_accuracy >= 0.99: score = 10
    elif combined_accuracy >= 0.98: score = 9
    elif combined_accuracy >= 0.97: score = 8
    elif combined_accuracy >= 0.96: score = 7
    elif combined_accuracy >= 0.95: score = 6
    elif combined_accuracy >= 0.90: score = 5
    else: score = 4
    return score


if __name__ == '__main__':
    program_start = time.time()
    combined_accuracy, classification_accuracy, bbox_score, segmentation_accuracy = evaluate()
    score = calculate_score(combined_accuracy)

    program_end = time.time()
    total_time = round(program_end - program_start, 3)
    
    print("\nDone!")
    print("Execution time (seconds) = ", total_time)
    print("Score = ", score, 
         "\nCombined accuracy = ", combined_accuracy, 
         "\nClassification accuracy = ", classification_accuracy,
         "\nbbox score = ", bbox_score,
         "\nSegmentation Accuracy = ", segmentation_accuracy)

1. Loading Data...
ImageDataBunch;

Train: LabelList (754 items)
x: SegmentationItemList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: SegmentationLabelList
ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256)
Path: /content/gdrive/My Drive/Colab Notebooks/deep_learning/bbc_train/images;

Valid: LabelList (188 items)
x: SegmentationItemList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: SegmentationLabelList
ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256)
Path: /content/gdrive/My Drive/Colab Notebooks/deep_learning/bbc_train/images;

Test: None
2. Instantiating Model...
3. Evaluating on Test Images...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Done!
Execution time (seconds) =  56.462
Score =  10 
Combined accuracy =  0.993008932021136 
Classification accuracy =  1.0 
bbox score =  0.9860916153993458 
Segmentation Accuracy =  0.9929351806640625


In [22]:
learn = cnn_learner(data, models.resnet18, custom_head=Network(), loss_func=Loss(), metrics=[class_pred_accuracy, bbox_accuracy, pixel_accuracy])

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
learn.freeze()

In [24]:
lr=1e-2
learn.fit_one_cycle(16, slice(lr/10, lr))

epoch,train_loss,valid_loss,class_pred_accuracy,bbox_accuracy,pixel_accuracy,time
0,14.045419,8.951955,0.957447,8.757220,0.977785,00:18
1,14.366639,8.581291,0.973404,8.378757,0.963591,00:18
2,14.623668,8.509036,0.962766,8.163036,0.942793,00:17
3,14.407997,8.548100,0.952128,8.249640,0.962397,00:18
4,14.304799,8.305079,0.978723,8.121021,0.975459,00:17
5,13.968884,7.232319,0.968085,7.076755,0.975593,00:18
6,13.573777,8.284464,0.957447,8.031461,0.976302,00:18
7,13.407240,11.555298,0.984043,11.390744,0.977175,00:18
8,13.140769,9.360056,0.984043,9.188896,0.977150,00:18
9,12.637105,6.022933,0.968085,5.843125,0.982418,00:18


In [ ]:
learn.unfreeze()

In [ ]:
learn.export('/content/gdrive/MyDrive/Colab Notebooks/deep_learning/Assignment/export.pkl')

In [ ]:
model = Model()

In [ ]:
img = cv2.imread('/content/gdrive/MyDrive/Colab Notebooks/deep_learning/bbc_train/images/P2050742.jpg')
